In [ ]:
import pandas as pd
songs_df = pd.read_pickle('/content/drive/MyDrive/MLProject/song-embeddings.pkl')
lyric_df = pd.read_pickle('/content/drive/MyDrive/MLProject/lyrics-embeddings-final.pkl')

In [ ]:
import ast

In [ ]:
lyric_df['ast_embd'] = songs_df['ast_embd']

In [ ]:
df=lyric_df

In [ ]:
df.iloc[0]['text_embd_1'].shape

(1, 1024)

In [ ]:
df.iloc[0]['ast_embd'].shape

(1, 768)

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
import ast
# df['merged_labels'] = df['merged_labels'].apply(ast.literal_eval)
all_genres=['electronic', 'rock', 'funk&soul', 'pop', 'folk&country', 'hiphop', 'rhythm&blues', 'jazz']
all_moods = ['happy/celebration/party/dance', 'loss/sad/heartbreak/angst/protest', 'romantic/love/passion/devotion', 'motivating/inspirational/uplifting/confidence/nostalgia']
mlb = MultiLabelBinarizer(classes=all_genres + all_moods)
df['labels'] = list(mlb.fit_transform(df['merged_labels']))
labels = mlb.classes_

In [ ]:
print (labels)

['electronic' 'rock' 'funk&soul' 'pop' 'folk&country' 'hiphop'
 'rhythm&blues' 'jazz' 'happy/celebration/party/dance'
 'loss/sad/heartbreak/angst/protest' 'romantic/love/passion/devotion'
 'motivating/inspirational/uplifting/confidence/nostalgia']


In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.labels = list(dataframe['labels'])
        self.text_embeddings = list(dataframe['text_embd_1'])
        self.ast_embeddings = list(dataframe['ast_embd'])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            'text_embedding': torch.tensor(self.text_embeddings[idx], dtype=torch.float),
            'ast_embedding': torch.tensor(self.ast_embeddings[idx], dtype=torch.float),
            'labels': torch.tensor(self.labels[idx], dtype=torch.float)
        }

# Create datasets
train_dataset = CustomDataset(train_df)
val_dataset = CustomDataset(val_df)
test_dataset = CustomDataset(test_df)

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)
test_loader = DataLoader(test_dataset, batch_size=16)

In [ ]:
import torch

In [ ]:
from torch import nn

class CustomClassificationHead(nn.Module):
    def __init__(self, roberta_hidden_size, ast_hidden_size, num_labels):
        super(CustomClassificationHead, self).__init__()
        self.total_hidden_size = roberta_hidden_size + ast_hidden_size
        self.layernorm = nn.LayerNorm(self.total_hidden_size)
        self.dense = nn.Linear(self.total_hidden_size, self.total_hidden_size)
        self.dropout = nn.Dropout(0.1)
        self.out_proj = nn.Linear(self.total_hidden_size, num_labels)

    def forward(self, roberta_embedding, ast_embedding):
        combined_embedding = torch.cat((roberta_embedding, ast_embedding), dim=1)
        normed_embedding = self.layernorm(combined_embedding)
        x = self.dropout(normed_embedding)
        x = self.dense(x)
        x = torch.tanh(x)
        x = self.dropout(x)
        x = self.out_proj(x)
        return x

In [ ]:
from torch.nn import BCEWithLogitsLoss
from transformers import AdamW
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss

loss_fn = BCEWithLogitsLoss()
epochs=30

In [ ]:
def exact_match_accuracy(true_labels, predictions):
    exact_match = np.all(true_labels == predictions, axis=1)
    ema_accuracy = np.mean(exact_match)
    genre_true, mood_true = true_labels[:, :8], true_labels[:, 8:]
    genre_pred, mood_pred = predictions[:, :8], predictions[:, 8:]
    exact_match_genre = np.all(genre_true == genre_pred, axis=1)
    exact_match_mood = np.all(mood_true == mood_pred, axis=1)
    exact_match_genres = np.mean(exact_match_genre)
    exact_match_moods = np.mean(exact_match_mood)
    return ema_accuracy, exact_match_genres, exact_match_moods

In [ ]:
# Initialize the custom classification head
custom_head = CustomClassificationHead(1024, 768, len(mlb.classes_))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_head.to(device)

# Loss function and optimizer
loss_fn = BCEWithLogitsLoss()
optimizer = AdamW(custom_head.parameters(), lr=6e-5)

for epoch in range(epochs):
    custom_head.train()
    total_train_loss = 0

    for batch in train_loader:
        optimizer.zero_grad()
        text_emb = batch['text_embedding'].to(device)
        ast_emb = batch['ast_embedding'].to(device)
        labels = batch['labels'].to(device)

        outputs = custom_head(text_emb.squeeze(1), ast_emb.squeeze(1))
        loss = loss_fn(outputs, labels)
        total_train_loss += loss.item()
        loss.backward()
        optimizer.step()

    avg_train_loss = total_train_loss / len(train_loader)
    print(f'Epoch {epoch+1} - Average Training Loss: {avg_train_loss:.4f}')

    # Validation loop
    custom_head.eval()
    total_val_loss = 0
    val_predictions = []
    val_true_labels = []

    with torch.no_grad():
        for batch in val_loader:
            text_emb = batch['text_embedding'].to(device)
            ast_emb = batch['ast_embedding'].to(device)
            labels = batch['labels'].to(device)

            outputs = custom_head(text_emb.squeeze(1), ast_emb.squeeze(1))
            loss = loss_fn(outputs, labels)
            total_val_loss += loss.item()

            val_predictions.append(torch.sigmoid(outputs).cpu().numpy())
            val_true_labels.append(labels.cpu().numpy())

    avg_val_loss = total_val_loss / len(val_loader)
    print(f'Epoch {epoch+1} - Average Validation Loss: {avg_val_loss:.4f}')

    # Calculate metrics
    val_predictions = np.vstack(val_predictions)
    val_true_labels = np.vstack(val_true_labels)
    threshold = 0.5
    val_predictions[val_predictions >= threshold] = 1
    val_predictions[val_predictions < threshold] = 0

    val_f1 = f1_score(val_true_labels, val_predictions, average='micro')
    val_precision = precision_score(val_true_labels, val_predictions, average='micro')
    val_recall = recall_score(val_true_labels, val_predictions, average='micro')
    val_hamming = hamming_loss(val_true_labels, val_predictions)
    ema_score, genre_score, mood_score = exact_match_accuracy(val_true_labels, val_predictions)

    print(f'Epoch {epoch+1} - Validation F1 Score: {val_f1:.4f}')
    print(f'Epoch {epoch+1} - Validation Precision: {val_precision:.4f}')
    print(f'Epoch {epoch+1} - Validation Recall: {val_recall:.4f}')
    print(f'Epoch {epoch+1} - Validation Hamming Loss: {val_hamming:.4f}')
    print(f"Exact Match Accuracy: {ema_score:.4f}")
    print(f"Exact genre Match Accuracy: {genre_score:.4f}")
    print(f"Exact mood Match Accuracy: {mood_score:.4f}")
    print ('-'*30)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1 - Average Training Loss: 0.2171
Epoch 1 - Average Validation Loss: 0.2155
Epoch 1 - Validation F1 Score: 0.7968
Epoch 1 - Validation Precision: 0.8625
Epoch 1 - Validation Recall: 0.7404
Epoch 1 - Validation Hamming Loss: 0.0784
Exact Match Accuracy: 0.4706
Exact genre Match Accuracy: 0.5789
Exact mood Match Accuracy: 0.8142
------------------------------
Epoch 2 - Average Training Loss: 0.0931
Epoch 2 - Average Validation Loss: 0.2130
Epoch 2 - Validation F1 Score: 0.8127
Epoch 2 - Validation Precision: 0.8697
Epoch 2 - Validation Recall: 0.7627
Epoch 2 - Validation Hamming Loss: 0.0730
Exact Match Accuracy: 0.4954
Exact genre Match Accuracy: 0.6006
Exact mood Match Accuracy: 0.8204
------------------------------
Epoch 3 - Average Training Loss: 0.0733
Epoch 3 - Average Validation Loss: 0.2202
Epoch 3 - Validation F1 Score: 0.8119
Epoch 3 - Validation Precision: 0.8662
Epoch 3 - Validation Recall: 0.7640
Epoch 3 - Validation Hamming Loss: 0.0735
Exact Match Accuracy: 0.4861
Ex

In [ ]:
# Set the model to evaluation mode
custom_head.eval()
from sklearn.metrics import f1_score, precision_score, recall_score, hamming_loss

# Initialize variables to store results
test_loss = 0
test_predictions = []
test_true_labels = []

# Loss function
loss_fn = BCEWithLogitsLoss()

# No gradient is needed for evaluation
with torch.no_grad():
    for batch in test_loader:

        text_emb = batch['text_embedding'].to(device)
        ast_emb = batch['ast_embedding'].to(device)
        labels = batch['labels'].to(device)

        outputs = custom_head(text_emb.squeeze(1), ast_emb.squeeze(1))
        loss = loss_fn(outputs, labels)
        test_loss += loss.item()

        # Store predictions and true labels
        test_predictions.append(torch.sigmoid(outputs).cpu().numpy())
        test_true_labels.append(labels.cpu().numpy())

# Calculate average loss
average_test_loss = test_loss / len(test_loader)
print(f"Average Test Loss: {average_test_loss:.4f}")

# Flatten the outputs
test_predictions = np.vstack(test_predictions)
test_true_labels = np.vstack(test_true_labels)

# Convert probabilities to binary predictions
threshold = 0.5
test_predictions[test_predictions >= threshold] = 1
test_predictions[test_predictions < threshold] = 0

# Calculate metrics
f1 = f1_score(test_true_labels, test_predictions, average='micro')
precision = precision_score(test_true_labels, test_predictions, average='micro')
recall = recall_score(test_true_labels, test_predictions, average='micro')
f1_mac = f1_score(test_true_labels, test_predictions, average='macro')
precision_mac = precision_score(test_true_labels, test_predictions, average='macro')
recall_mac = recall_score(test_true_labels, test_predictions, average='macro')
f1_w = f1_score(test_true_labels, test_predictions, average='weighted')
precision_w = precision_score(test_true_labels, test_predictions, average='weighted')
recall_w = recall_score(test_true_labels, test_predictions, average='weighted')
hamming = hamming_loss(test_true_labels, test_predictions)
ema_score, genre_score, mood_score = exact_match_accuracy(test_true_labels, test_predictions)
print(f"F1 Score: {f1:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("-"*30)
print(f"Hamming Loss: {hamming:.4f}")
print(f"F1 Score Macro: {f1_mac:.4f}")
print(f"Precision Macro: {precision_mac:.4f}")
print(f"Recall Macro: {recall_mac:.4f}")
print("-"*30)
print(f"F1 Score W: {f1_w:.4f}")
print(f"Precision W: {precision_w:.4f}")
print(f"Recall W: {recall_w:.4f}")
print(f"Exact Match Accuracy: {ema_score:.4f}")
print(f"Exact Match Accuracy genre: {genre_score:.4f}")
print(f"Exact Match Accuracy mood: {mood_score:.4f}")


Average Test Loss: 0.2324
F1 Score: 0.8558
Precision: 0.8653
Recall: 0.8465
------------------------------
Hamming Loss: 0.0571
F1 Score Macro: 0.8662
Precision Macro: 0.8787
Recall Macro: 0.8561
------------------------------
F1 Score W: 0.8560
Precision W: 0.8674
Recall W: 0.8465
Exact Match Accuracy: 0.6262
Exact Match Accuracy genre: 0.7426
Exact Match Accuracy mood: 0.8094
